In [38]:
import igraph
import numpy as np

In [39]:
g = igraph.Graph.Read_Pickle('ogbn-arxiv.pickle')

## Task 1

In [40]:
print('number of vertices:', g.vcount())
print('number of edges:', g.ecount())

number of vertices: 169343
number of edges: 1166243


In [61]:
# get list of in degree for each node (= number citations)
indegree_list = g.indegree()

# get ids of 5 most cited articles
most_cited = np.argsort(indegree_list)[-5:]

# print their subject areas
print('subjects of 5 most cited articles:')
for id in most_cited:
    print(*g.vs['label'][id])

subjects of 5 most cited articles:
16
24
16
16
24


In [62]:
# get nodes with an edge to the most cited articles (= articles citing the most cited ones)
citing_ids = []
for id in most_cited:
    citing_ids += g.predecessors(id)

# remove any duplicates
citing_ids = np.unique(citing_ids)

# get the subject area of each article
citing_subj = np.array(g.vs['label'])[citing_ids]

# count occurence of each subject area
subjects, counts = np.unique(citing_subj, return_counts=True)

# print subject area with maximum occurrences
print('subject area that most articles referencing the 5 most cited articles belong to:', subjects[np.argmax(counts)])

subject area that most articles referencing the 5 most cited articles belong to: 16
